In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
ratings = pd.DataFrame.from_csv('Small/ratings.csv',index_col=None)
movies = pd.DataFrame.from_csv('Small/movies.csv',index_col='movieId')

In [3]:
#Step 1: Remove movies prior to August 2003 and with less ratings than the required threshold
r = ratings
r = r[r.timestamp >= 1059696000] #1059696000 is the UTCTimestamp for Aug 1, 2003 at GMT
r = r.set_index('userId')
r['user_freq'] = r.index.value_counts() #Generate new column to filter data by: Number of ratings by the user
r = r[(r.user_freq <= 2000)&(r.user_freq >= 40)] #Weed out suspect users and those with too few ratings. Lower threshold chosen to ensure user presence in all data partitions.
r = r.reset_index()
r = r.set_index('movieId') #Generate new column to filter data by: number of ratings per movie
r['movie_freq'] = r.index.value_counts()
r = r[r.movie_freq >=35] #Filter out movies with too few ratings. Threshold chosen to ensure user presence in all data partitions.
r = r.reset_index()
r = r.drop(['user_freq','movie_freq','timestamp'],axis=1) #Remove excess data

r.shape

(20816, 3)

In [4]:
#Ensuring that the train/test partition produces datasets that pivot to the same dimensions (include all movies and all users).
r_piv = r.pivot('movieId','userId','rating')
same_shape = False
while same_shape == False:
    train, test = train_test_split(r, train_size = 0.80) #Randomly partitions data into 80-20 training-test split
    trainm = train.pivot('movieId','userId','rating')
    testm = test.pivot('movieId','userId','rating')
    if (trainm.shape == r_piv.shape) & (testm.shape == r_piv.shape):
        same_shape = True

k = 4 #Setting k for k-fold cross validation of the training partition
n = int(len(train)/k) #Establishing size of each fold (except last) for k-fold cross validation

ind = r_piv.index #Storing movieId index for future use
col = r_piv.columns #Storing userId column names fof future use
nmovies, nusers = r_piv.shape #Store dimensions of the data
mu = r_piv.mean(axis = 1) #Store average rating per movie: used in mean normalization

In [5]:
#Ensuring that all k attempts at k-fold validation will use matrices with matching dimensions
dim_check = False
while dim_check == False:
    shuffled_train = train.reindex(np.random.permutation(train.index)) #Randomize order of data (long form)
    for num in range(0,k):
        #Establish dataset for cross-validation for each k
        if num != k-1:
            cv = shuffled_train[num*n:(num+1)*n]
        else: #Final k may contain different number of entries due to remainder when calculating n
            cv = shuffled_train[num*n:]
        tr = shuffled_train.drop(cv.index) #Training partition without cross-validation: used to train model for each k
        trm = tr.pivot('movieId','userId','rating')
        cvm = cv.pivot('movieId','userId','rating')
        
        #if the dimensions of data matrix for all k values don't match, rerandomize long form data, split, and try again
        if (trm.shape != trainm.shape) | (cvm.shape != trainm.shape):
            dim_check = False
            break #exits for loop early if first few k values don't have matching dimensions
        else:
            dim_check = True

In [6]:
Results = []
for trial in range(1,4): #Three trials to account for random nature of feature initialization
    for nfeatures in [200,250,300,350,400]: #For loop to optimize for number of features used in final models
        for reg in [2.5,5,10]: #For loop to optimize the regularization parameter used in final models
            alpha = .003 #learning rate
            cum_red = 0 #Stores cumulative reduction in RMSE across all k's when doing cross-validation
            for num in range(0,k): #For loop for k-fold cross validation
                X = np.random.randn(nmovies,nfeatures) #Random initialization of movie features
                Theta = np.random.randn(nusers,nfeatures) #Random initialization of user features
                
                #Build training and cross-validation matrices for this k
                if num != k-1:
                    cv = shuffled_train[num*n:(num+1)*n]
                else:
                    cv = shuffled_train[num*n:]
                tr = shuffled_train.drop(cv.index)
                trm = tr.pivot('movieId','userId','rating')
                cvm = cv.pivot('movieId','userId','rating')
                
                trm2 = trm.subtract(mu,axis=0) #Mean normalization: so that the regularization drives non-existant ratings towards the mean rating, rather than 0
                R = np.asarray(~np.isnan(trm2)) #Matrix containing boolean for rated/not rated for each movie user pairing
                trm2 = np.asarray(trm2.fillna(0)) #Replaces missing data with 0's

                delta = 1 #Stores change in each iteration's cost function
            
                J_new = np.sum(np.sum(np.multiply(((np.dot(X,np.transpose(Theta))-trm2)**2),R)))/2 + reg/2*(np.sum(np.sum(Theta**2))+np.sum(np.sum(X**2))) #First initialization of cost function
                while delta >= 0.001: #Iterates until cost function changes by < 0.1%
                    J_old = J_new
                    X_grad = np.dot(np.multiply(np.dot(X,np.transpose(Theta))-trm2,R),Theta) + reg*X #Determine gradient for all movie features
                    Theta_grad = np.dot(np.transpose(np.multiply(np.dot(X,np.transpose(Theta))-trm2,R)),X)+reg*Theta #Determine gradient for all user features
                    X = X-alpha*X_grad #Movie feature update
                    Theta = Theta-alpha*Theta_grad #User feature update
                    J_new = np.sum(np.sum(np.multiply(((np.dot(X,np.transpose(Theta))-trm2)**2),R)))/2 + reg/2*(np.sum(np.sum(Theta**2))+np.sum(np.sum(X**2))) #Recalculate cost function
                    delta = (J_old-J_new)/J_old #Determine change in cost function
                    if delta <0: #If cost function increased, undo the feature updates, reduce learning rate
                        X = X+alpha*X_grad #Undoes update to movie features
                        Theta = Theta+alpha*Theta_grad #Undoes update to user features
                        J_new = J_old
                        alpha = alpha/2 #Reduces learning rate by half
                        delta=1 #Ensures re-entry into the while loop
                
                Predictions = pd.DataFrame(data = np.dot(X,np.transpose(Theta)),index = ind, columns = col) #Generate normalized rating predictions: dot product of movie and user features
                Predictions = Predictions.add(mu,axis = 0) #Add back mean to undo mean normalization
            
                RMSE_avg = math.sqrt(np.sum(np.sum((cvm.subtract(mu,axis = 0))**2))/len(cv)) #RMSE of comparison model - give each user the movie's average rating
                RMSE_alg = math.sqrt(np.sum(np.sum((Predictions - cvm)**2))/len(cv)) #RMSE of currently trained model
                cum_red += (RMSE_avg - RMSE_alg)/RMSE_avg*100 #Adds % reduction in RMSE for this k to running total
            
            red = cum_red/k #Dividing by k gives the average % rduction in RMSE to be expected from this Trial, number of features, and regularization parameter
                        
            Results.append({'Trial': trial, 'Features': nfeatures, 'Reg. Parameter': reg, '% Reduction in RMSE': red}) #Store results for future inspection

Results = pd.DataFrame(Results)
Results = Results.set_index(['Trial','Features','Reg. Parameter'])
Results = Results.unstack('Trial')

In [7]:
Results

% Reduction in RMSE                    
Trial                                     1         2         3
Features Reg. Parameter                                        
200      2.5                       5.589384  5.535174  5.445152
         5.0                       6.840531  6.812381  6.725516
         10.0                      5.621169  5.664231  5.618512
250      2.5                       5.986714  6.036809  6.181394
         5.0                       6.849052  6.789370  6.877346
         10.0                      5.616131  5.642231  5.636288
300      2.5                       6.253209  6.111578  6.361749
         5.0                       6.972623  7.013077  6.915094
         10.0                      5.688899  5.635335  5.635817
350      2.5                       5.305559  5.073112  5.030704
         5.0                       6.324296  6.313873  6.394893
         10.0                      5.173953  5.226618  5.183267
400      2.5                       5.448427  5.462609  5.276864
         5.0                       6.242627  6.370354  6.413241
         10.0                      5.147260  5.249160  5.223585

In [8]:
avg_red = np.mean(Results,axis=1) #averaging results across all Trials
avg_red

Features  Reg. Parameter
200       2.5               5.523237
          5.0               6.792809
          10.0              5.634637
250       2.5               6.068306
          5.0               6.838589
          10.0              5.631550
300       2.5               6.242179
          5.0               6.966931
          10.0              5.653350
350       2.5               5.136458
          5.0               6.344354
          10.0              5.194613
400       2.5               5.395967
          5.0               6.342074
          10.0              5.206668
dtype: float64

In [9]:
nfeatures, reg = np.argmax(avg_red) #Determine best pairing of regularization parameter and number of features
nfeatures, reg

(300, 5.0)

In [10]:
trainm2 = trainm.subtract(mu, axis = 0) #Mean normalization of 80% data partition
R = np.asarray(~np.isnan(trainm2)) #Matrix containing boolean for rated/not rated for each movie user pairing
trainm2 = np.asarray(trainm2.fillna(0)) #Replace missing data with 0's

Results = []
for num in range(0,5): #Due to random initializations leading to local minima, we need 5 trials so we can report a more accurate % reduction in RMSE
    X = np.random.randn(nmovies,nfeatures) #Random initialization of movie features
    Theta = np.random.randn(nusers,nfeatures) #Random initialization of user features

    alpha = .003 #learning rate

    delta = 1 #Stores change in each iteration's cost function
    iter = 0 #In case we wish to visualize the decaying magnitude of the cost-function across the upcoming iterations
    J = []
    J_new = np.sum(np.sum(np.multiply(((np.dot(X,np.transpose(Theta))-trainm2)**2),R)))/2 + reg/2*(np.sum(np.sum(Theta**2))+np.sum(np.sum(X**2))) #First initialization of cost function
    J.append({'Iteration': iter, 'Cost': J_new}) #In case we wish to visualize the decaying magnitude of the cost-function across the upcoming iterations
    while delta >= 0.001:
        J_old = J_new
        iter = iter+1
        X_grad = np.dot(np.multiply(np.dot(X,np.transpose(Theta))-trainm2,R),Theta) + reg*X #Determine gradients for movie features
        Theta_grad = np.dot(np.transpose(np.multiply(np.dot(X,np.transpose(Theta))-trainm2,R)),X)+reg*Theta #Determine gradients for user features
        X = X-alpha*X_grad #Movie feature update
        Theta = Theta-alpha*Theta_grad #User feature update
        J_new = np.sum(np.sum(np.multiply(((np.dot(X,np.transpose(Theta))-trainm2)**2),R)))/2 + reg/2*(np.sum(np.sum(Theta**2))+np.sum(np.sum(X**2))) #Recalculating cost function after the update
        J.append({'Iteration': iter, 'Cost': J_new}) #In case we wish to visualize the decaying magnitude of the cost-function across the upcoming iterations
        delta = (J_old-J_new)/J_old #Determine change in cost function
        if delta <0: #If cost function increased, undo the feature updates, reduce learning rate
            X = X+alpha*X_grad
            Theta = Theta+alpha*Theta_grad
            J_new = J_old
            alpha = alpha/2
            delta=1
                
    Predictions = pd.DataFrame(data = np.dot(X,np.transpose(Theta)),index = ind, columns = col) #Generate normalized rating predictions: dot product of movie and user features
    Predictions = Predictions.add(mu,axis = 0) #Add back mean to undo mean normalization
    
    n_obs= len(testm)        
    RMSE_avg = math.sqrt(np.sum(np.sum((testm.subtract(mu,axis = 0))**2))/n_obs) #RMSE of comparison model - give each user the movie's average rating
    RMSE_alg = math.sqrt(np.sum(np.sum((Predictions - testm)**2))/n_obs) #RMSE of currently trained model
    red = (RMSE_avg - RMSE_alg)/RMSE_avg*100 #% reduction in RMSE for this trial
            
    Results.append(red) #Store results for future inspection

Results

[7.322327762639662,
 7.496057426963909,
 7.559253376952613,
 7.316127679573875,
 7.358343049990447]

In [11]:
np.mean(Results) #Average % Reduction in RMSE for reporting purposes

7.4104218592241011

In [12]:
#PCA on movie features: used to make features linearly uncorrelated. Also results in dimensionality reduction and data compression.
var_explained = 0
iter = 0
while var_explained < 0.95: #Want to retain 95% of variance in movie features
    iter += 1 #Increase number of components
    pca = PCA(n_components=iter)
    movie_pca = pd.DataFrame(pca.fit_transform(X), index = ind) #Determine and store principal components for this iteration
    var_explained = np.sum(pca.explained_variance_ratio_) #Determine how much of the variance was retained after PCA

#PCA on user features: used to make features linearly uncorrelated. Also results in dimensionality reduction and data compression. Goes unused for now.
var_explained = 0
iter = 0
while var_explained < 0.95:
    iter += 1
    pca = PCA(n_components=iter)
    user_pca = pd.DataFrame(pca.fit_transform(Theta), index = col)
    var_explained = np.sum(pca.explained_variance_ratio_)

In [13]:
#Normalize movie features from 0 to 1, a.k.a. Feature Scaling
movie_pca_norm = (movie_pca-movie_pca.min(axis = 0))/(movie_pca.max(axis = 0) - movie_pca.min(axis =0))

#Store Normalized Euclidean distance from every other movies based on principal components of the learned features
dist = []
for num1 in range(0,nmovies):
    for num2 in range(0,nmovies):
        dist.append({'movieId1': ind[num1], 'movieId2': ind[num2], 'distance':math.sqrt(np.sum((movie_pca_norm[movie_pca_norm.index == ind[num1]].values-movie_pca_norm[movie_pca_norm.index == ind[num2]].values)**2))})

#Establish easily queriable collection of distances
d = pd.DataFrame(dist)
d = d.set_index(['movieId1','movieId2'])
d = d[d.distance != 0] #Removes pairing that contains 2 instances of the same movie

In [14]:
d_mov = 4896 #movieId from example (Harry Potter and the Philosopher's Stone)
closest = d.loc[d_mov].sort_values('distance').head(10) #Find 10 'nearest' movies, only contains movieId
print("Movies closest to %s:" %movies.loc[d_mov].title)
closest = pd.merge(closest,movies,left_index = True, right_index = True).sort_values('distance') #Add title to make the recommendation clearer
closest

Movies closest to Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001):


,distance,title,genres
551,2.045114,"Nightmare Before Christmas, The (1993)",Animation|Children|Fantasy|Musical
40815,2.065060,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller|IMAX
5816,2.065709,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
68237,2.093137,Moon (2009),Drama|Mystery|Sci-Fi|Thriller
3000,2.097690,Princess Mononoke (Mononoke-hime) (1997),Action|Adventure|Animation|Drama|Fantasy
1127,2.104640,"Abyss, The (1989)",Action|Adventure|Sci-Fi|Thriller
1617,2.111250,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
47610,2.125300,"Illusionist, The (2006)",Drama|Fantasy|Mystery|Romance
7458,2.133924,Troy (2004),Action|Adventure|Drama|War
2001,2.139426,Lethal Weapon 2 (1989),Action|Comedy|Crime|Drama


In [15]:
my_ratings = pd.DataFrame.from_csv('My Ratings.csv',index_col='movieId') #Loads my movie ratings
my_rat_mean_norm = my_ratings.subtract(mu,axis = 0) #Mean normalization of my ratings
R = np.asarray(~np.isnan(my_rat_mean_norm))
my_rat_mean_norm = np.asarray(my_rat_mean_norm.fillna(0)) #Replaces missing ratings with 0's
my_features = np.random.randn(1,nfeatures) #Random initialization of my user features

In [16]:
alpha = .003 #learning rate for upcoming linear regression

#Optimizing linear regression to learn my user features.
#No trials required because regardless of the initialization, the result should be a global minimum for that regularization parameter.
best_RMSE = 100 #Use RMSE to determine best fit
for reg in [1,2.5,5,10]:
    delta = 1
    J_new = np.sum(np.multiply(((np.dot(X,np.transpose(my_features))-my_rat_mean_norm)**2),R))/2 + reg/2*np.sum(my_features**2) #Linear regression cost function
    while delta >= 0.001:
        J_old = J_new
        iter = iter+1
        my_features_grad = np.dot(np.transpose(np.multiply(np.dot(X,np.transpose(my_features))-my_rat_mean_norm,R)),X) + reg*my_features #Gradient for my features
        my_features = my_features - alpha*my_features_grad #Update my user features
        J_new = np.sum(np.multiply(((np.dot(X,np.transpose(my_features))-my_rat_mean_norm)**2),R))/2 + reg/2*np.sum(my_features**2) #Recalculate cost function
        delta = (J_old-J_new)/J_old
    my_predictions = pd.DataFrame(data = np.dot(X,np.transpose(my_features)),index = ind) #Generate normalized predictions based on my features
    my_predictions = my_predictions.add(mu,axis = 0) #Re-add mean to undo mean normalization
    RMSE = np.sqrt(np.sum((my_predictions-my_ratings.values)**2)/np.sum(R)).values #Determine RMSE to compare against other models
    #Store best regularization parameter
    if RMSE < best_RMSE:
        best_RMSE = RMSE
        best_reg = reg

#Re-determine my features using the best regularization parameter
reg = best_reg
delta = 1
J_new = np.sum(np.multiply(((np.dot(X,np.transpose(my_features))-my_rat_mean_norm)**2),R))/2 + reg/2*np.sum(my_features**2) #Linear regression cost function
while delta >= 0.001:
    J_old = J_new
    iter = iter+1
    my_features_grad = np.dot(np.transpose(np.multiply(np.dot(X,np.transpose(my_features))-my_rat_mean_norm,R)),X) + reg*my_features #Gradient for my features
    my_features = my_features - alpha*my_features_grad #Update my user features
    J_new = np.sum(np.multiply(((np.dot(X,np.transpose(my_features))-my_rat_mean_norm)**2),R))/2 + reg/2*np.sum(my_features**2) #Recalculate cost function
    delta = (J_old-J_new)/J_old
my_predictions = pd.DataFrame(data = np.dot(X,np.transpose(my_features)),index = ind,columns=['Predicted Rating']) #Generate normalized predictions based on my features
my_predictions = my_predictions.add(mu,axis = 0).merge(movies,left_index = True, right_index = True) #Re-add mean to undo mean normalization and add in movie titles
my_predictions.sort_values(by='Predicted Rating',ascending=False).head(10) #Sort my predicted ratings in descending order

,Predicted Rating,title,genres
movieId,,,
318,5.237571,"Shawshank Redemption, The (1994)",Crime|Drama
2959,5.116613,Fight Club (1999),Action|Crime|Drama|Thriller
50,5.106159,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
2571,5.032582,"Matrix, The (1999)",Action|Sci-Fi|Thriller
48516,5.015751,"Departed, The (2006)",Crime|Drama|Thriller
293,4.956526,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
296,4.902465,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
58559,4.837117,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
3000,4.829178,Princess Mononoke (Mononoke-hime) (1997),Action|Adventure|Animation|Drama|Fantasy
